In [22]:
import pandas as pd
from sklearn.linear_model import LinearRegression
regressor = LinearRegression()

df = pd.read_excel("real_estate_data.xlsx",'Real_Estate_data')
print (df.count())

df.corr() #get the correlation matrix to see to which level the dataset attributes are correlated, that will affect selecting attributes in building the forecasting model
# the matrix inform us that there is a strong relationship between pricem age, area, and floor
#df.corr()['SalePrice'] #to get correlation for specific field with other atributes

SalePrice      5891
BuildingAge    5891
Area           5891
Floor          5891
Location       5891
dtype: int64


,SalePrice,BuildingAge,Area,Floor,Location
SalePrice,1.000000,0.447875,0.697199,0.336729,0.010615
BuildingAge,0.447875,1.000000,0.021282,0.260568,0.005180
Area,0.697199,0.021282,1.000000,0.179491,0.017981
Floor,0.336729,0.260568,0.179491,1.000000,0.005355
Location,0.010615,0.005180,0.017981,0.005355,1.000000


In [23]:
#selection to attributes where according to the correlation
Xtest = df[['BuildingAge','Area','Floor','Location']] 
X = df[['BuildingAge','Area','Floor']]
y = df[['SalePrice']]

from sklearn.feature_selection import RFE

rfe = RFE(regressor,n_features_to_select=3,step=43) # you can tell the eliminator how many features you need
fit = rfe.fit(Xtest, y) # let him calculate it free way
print("Num Features: %d" % fit.n_features_)
print("Selected Features: %s" % fit.support_) #False means never to exclude it
print("Feature Ranking: %s" % fit.ranking_) #the more value the more will impact your model

for i in range(Xtest.shape[1]):
	print('Column: %d, Selected %s, Rank: %.3f' % (i, rfe.support_[i], rfe.ranking_[i]))

Num Features: 3
Selected Features: [ True False  True  True]
Feature Ranking: [1 2 1 1]
Column: 0, Selected True, Rank: 1.000
Column: 1, Selected False, Rank: 2.000
Column: 2, Selected True, Rank: 1.000
Column: 3, Selected True, Rank: 1.000


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [24]:
from sklearn.feature_selection import chi2
from sklearn.feature_selection import SelectKBest
chiSelector = SelectKBest(chi2,k=3)
X_kbest = chiSelector.fit(Xtest, y)
print(X_kbest.scores_)#We choose the highest values, which will be the first 3,selects the features according to the k highest score

[1.18453147e+02 5.61716913e+05 9.62733118e+03 1.10589874e+02]


In [49]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.65)
regressor.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)

In [50]:
print (regressor.predict([[1995,280,10]]))#prediction for apartment, with building age 1995, area 280, and floor 10

[[54911.14091451]]


In [51]:
#Model Evaluation
from sklearn import metrics
y_pred = regressor.predict(X_test)
#Model Evaluation
print('Mean Absolute Error (MAE):', metrics.mean_absolute_error(y_test, y_pred))
print('Mean Squared Error (MSE):', metrics.mean_squared_error(y_test, y_pred))
print('Explained Variance Score:', metrics.explained_variance_score(y_test, y_pred))
print('Mean Squared Log Error:', metrics.mean_squared_log_error(y_test, y_pred))
print('Median Absolute Error:', metrics.median_absolute_error(y_test, y_pred))
print('R^2:', metrics.r2_score(y_test, y_pred)) #Check the accuracty when changing the sample size

Mean Absolute Error (MAE): 48632.41508676349
Mean Squared Error (MSE): 3549000357.6350846
Explained Variance Score: 0.6843571324740131
Mean Squared Log Error: 0.09840001667540854
Median Absolute Error: 43866.12920816522
R^2: 0.683966841227774


# import pickle and forecasting from saved model

In [6]:
import pickle
pkl_filename = "pickle_model.pkl"
with open(pkl_filename, 'wb') as file:
    pickle.dump(regressor, file)

In [7]:
with open(pkl_filename, 'rb') as file:
    pickle_model = pickle.load(file)
    
Ypredict = pickle_model.predict([[2022,400,15]])
Ypredict

array([[212012.90861885]])

In [8]:
Ypredict= pickle_model.predict([[2022,300,10]])
Ypredict

array([[186280.8349495]])